In [4]:
import numpy as np
import pandas as pd

In [5]:
book = pd.read_csv("Data/Books.csv")
rating = pd.read_csv("Data/Ratings.csv")
users = pd.read_csv("Data/Users.csv")

C:\Users\rawat\AppData\Local\Temp\ipykernel_3696\2350495040.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv("Data/Books.csv")


In [6]:
book.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [14]:
rating.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [16]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


## Checking for missing values

In [17]:
#book data
book.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [18]:
#user data
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

- Age is Missing in most of the data

In [19]:
#rating data
rating.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

## Duplicate data

In [22]:
data  = [book, users, rating]
for i in data:
    print(i.duplicated().sum())

0
0
0


### Popularity Based Recommendation System

In [133]:
ratings_with_name = rating.merge(book, on="ISBN")
ratings_with_name.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...


In [134]:
num_rating_df = ratings_with_name.groupby("Book-Title").count()["Book-Rating"].reset_index()
num_rating_df.rename(columns={"Book-Rating":"Num-Rating"}, inplace=True)

In [147]:
avg_rating_df = ratings_with_name.groupby("Book-Title").mean(numeric_only=True,).round(2)["Book-Rating"].reset_index()
avg_rating_df.rename(columns = {"Book-Rating": "Avg-Rating"},inplace=True)

In [148]:
# Creating common df with both rating and num of data:
popular_df = num_rating_df.merge(avg_rating_df, on="Book-Title")
popular_df.head()

,Book-Title,Num-Rating,Avg-Rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.00


In [149]:
# Books with rating more than 250 and popular
top_50_df = popular_df[popular_df["Num-Rating"]>=250].sort_values("Avg-Rating",ascending=False).reset_index().head(50)
top_50_df

,index,Book-Title,Num-Rating,Avg-Rating
0,80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.85
1,80422,Harry Potter and the Goblet of Fire (Book 4),387,5.82
2,80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.74
3,80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.50
4,80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.18
5,191612,The Hobbit : The Enchanting Prelude to The Lor...,281,5.01
6,187377,The Fellowship of the Ring (The Lord of the Ri...,368,4.95
7,80445,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.90
8,211384,"The Two Towers (The Lord of the Rings, Part 2)",260,4.88
9,219741,To Kill a Mockingbird,510,4.70


In [151]:
# Creating a data frame have all the information of the book with their no. of votes and average rating:
top_50_df = top_50_df.merge(book, on="Book-Title").drop_duplicates("Book-Title")[["Book-Title","Book-Author","Image-URL-M","Num-Rating","Avg-Rating"]]
top_50_df

,Book-Title,Book-Author,Image-URL-M,Num-Rating,Avg-Rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.85
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.82
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.74
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.50
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.18
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.01
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.95
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.90
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.88
39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.70


### Collabrative base Recommendation System

In [138]:
ratings_with_name.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...


In [139]:
# find a User who has a minimum vote of 200 ( padhai likhaii users)
user_rating = ratings_with_name.groupby("User-ID").count().sort_values("Book-Rating",ascending=False)["ISBN"].reset_index()

In [79]:
padhe_likhe_user =user_rating[user_rating["ISBN"] > 200]["User-ID"].to_list()

In [83]:
# data of padhai likhai users
padhe_like_user_df = ratings_with_name[ratings_with_name["User-ID"].isin(padhe_likhe_user)]

In [92]:
# famous book read by padhai likhai users:
y = padhe_like_user_df.groupby("Book-Title").count()["Book-Rating"]>=50
famous_books = y[y].index

In [94]:
# final rating with user and books:
final_data = padhe_like_user_df[padhe_like_user_df["Book-Title"].isin(famous_books)]
final_data.shape

(58586, 10)

In [96]:
# Now making pivot table of user and books

pt = final_data.pivot_table(index="Book-Title",columns="User-ID",values="Book-Rating",fill_value=0)
pt.shape

(706, 810)

In [97]:
# Cosine similarities to find the similarities between the books.
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(pt)
similarity_scores.shape


(706, 706)

In [117]:
#Creating a function that recommends books
def recommend(book_name):
    #index fetching:
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key= lambda x:x[1], reverse=True)[1:9]
    for i in similar_items:
        print(pt.index[i[0]])

In [116]:
sorted(list(enumerate(similarity_scores[0])),key= lambda x:x[1], reverse=True)[1:9]

[(47, 0.2702651417103732),
 (545, 0.2639619371123497),
 (82, 0.23669374347400993),
 (634, 0.23299389358170394),
 (551, 0.2262639743141286),
 (181, 0.22222254415660225),
 (500, 0.21057743765309012),
 (344, 0.20675540390453995)]

In [109]:
np.where(pt.index=="A Bend in the Road")[0][0]

4

In [118]:
#Testing function
recommend("The Notebook")

A Walk to Remember
The Rescue
One Door Away from Heaven
Toxin
The Five People You Meet in Heaven
Nights in Rodanthe
Suzanne's Diary for Nicholas
Everything's Eventual : 14 Dark Tales


In [127]:
final_data["Image-URL-M"]

1150       http://images.amazon.com/images/P/002542730X.0...
1163       http://images.amazon.com/images/P/0060930535.0...
1165       http://images.amazon.com/images/P/0060934417.0...
1168       http://images.amazon.com/images/P/0061009059.0...
1174       http://images.amazon.com/images/P/006440188X.0...
                                 ...                        
1029196    http://images.amazon.com/images/P/1400031354.0...
1029197    http://images.amazon.com/images/P/1400031362.0...
1029270    http://images.amazon.com/images/P/1573229725.0...
1029309    http://images.amazon.com/images/P/1586210661.0...
1029310    http://images.amazon.com/images/P/1586212230.0...
Name: Image-URL-M, Length: 58586, dtype: object

In [170]:
top_50_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 195
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Book-Title   50 non-null     object 
 1   Book-Author  50 non-null     object 
 2   Image-URL-M  50 non-null     object 
 3   Num-Rating   50 non-null     int64  
 4   Avg-Rating   50 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.4+ KB


In [169]:
top_50_df.head(2)

,Book-Title,Book-Author,Image-URL-M,Num-Rating,Avg-Rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.85
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.82


In [171]:
import pickle
pickle.dump(top_50_df, open("popular.pkl","wb"))

In [167]:
top_50_df[top_50_df["Book-Title"]=="The Hitchhiker's Guide to the Galaxy"]

,Book-Title,Book-Author,Image-URL-M,Num-Rating,Avg-Rating
89,The Hitchhiker's Guide to the Galaxy,Douglas Adams,http://images.amazon.com/images/P/0671461494.0...,268,4.33


In [ ]:
https://epub.us/wp-content/uploads/2015/09/Hitchhikers-Guide-to-the-Galaxy.jpg

In [168]:
top_50_df.loc[89, "Image-URL-M"] = "https://epub.us/wp-content/uploads/2015/09/Hitchhikers-Guide-to-the-Galaxy.jpg"

In [172]:
top_50_df["Image-URL-M"][0]

'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'

In [204]:
# Making New Recommendation function which will return multiple data in one search.
# -> return -> Name, Auther, image-url
# pickle file required -> pt,final_data,similarity_score
def recommend2(book_name):
    #index fetching:
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key= lambda x:x[1], reverse=True)[1:9]
    data = []
    for i in similar_items:
        item = []
        temp_df = final_data[final_data["Book-Title"] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates("Book-Title")["Book-Title"].values))
        item.extend(list(temp_df.drop_duplicates("Book-Title")["Book-Author"].values))
        item.extend(list(temp_df.drop_duplicates("Book-Title")["Image-URL-M"].values))
        data.append(item)
        
    return data   

In [205]:
recommend2('Politically Correct Bedtime Stories: Modern Tales for Our Life and Times')

[["Don't Stand Too Close to a Naked Man",
  'Tim Allen',
  'http://images.amazon.com/images/P/0786889020.01.MZZZZZZZ.jpg'],
 ['Saving Faith',
  'David Baldacci',
  'http://images.amazon.com/images/P/0446608890.01.MZZZZZZZ.jpg'],
 ['Into Thin Air : A Personal Account of the Mt. Everest Disaster',
  'JON KRAKAUER',
  'http://images.amazon.com/images/P/0385492081.01.MZZZZZZZ.jpg'],
 ['Saint Maybe',
  'ANNE TYLER',
  'http://images.amazon.com/images/P/0804108749.01.MZZZZZZZ.jpg'],
 ['L Is for Lawless',
  'Sue Grafton',
  'http://images.amazon.com/images/P/0449221490.01.MZZZZZZZ.jpg'],
 ['Bag of Bones',
  'Stephen King',
  'http://images.amazon.com/images/P/067102423X.01.MZZZZZZZ.jpg'],
 ['Notes from a Small Island',
  'Bill Bryson',
  'http://images.amazon.com/images/P/0688147259.01.MZZZZZZZ.jpg'],
 ['Five Quarters of the Orange',
  'Joanne Harris',
  'http://images.amazon.com/images/P/0060958022.01.MZZZZZZZ.jpg']]

In [186]:
final_data.loc[1150, ["Image-URL-M","Book-Author","Book-Title"]].tolist()

['http://images.amazon.com/images/P/002542730X.01.MZZZZZZZ.jpg',
 'James Finn Garner',
 'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times']

In [200]:
final_data.shape

(58586, 10)

In [207]:
# pickle File related to function
pickle.dump(pt,open("pt.pkl","wb"))
pickle.dump(final_data,open("final_data.pkl","wb"))
pickle.dump(similarity_scores, open("similarity_score.pkl","wb"))